In [1]:
import torch 
import time 
from sklearn.model_selection import KFold

import torch.nn as nn
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from gensim.models.phrases import Phrases
from gensim.models import Word2Vec

from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.evaluation import classification_inference
from src.metric import  multi_cls_metrics,multi_cls_log
from src.dataset.tokenizer import GensimTokenizer
from src.enhancement.temporal import TemporalEnsemble

from iflytek_app.dataset import MixDataset
from iflytek_app.process import train_process, test_process, result_process, kfold_inference
from iflytek_app.models import TextcnnTemporal, TextcnnAugTemporal
device = get_torch_device()
set_seed()

No GPU available, using the CPU instead.


In [2]:
c2v = GensimTokenizer( Word2Vec.load('./checkpoint/char_min1_win5_sg_d100'))
w2v = GensimTokenizer(Word2Vec.load('./checkpoint/phrase_min1_win5_sg_d100'))
w2v.init_vocab()
c2v.init_vocab()
phraser = Phrases.load('./checkpoint/phrase_tokenizer')
df, label2idx = train_process()
test = test_process()
label2idx.update({'unlabel':-1})

                id           l1           l2          len
count  4199.000000  4199.000000  4199.000000  4199.000000
mean   2099.000000     8.969278    37.087878    46.057156
std    1212.291219     4.576621    79.204914    79.332999
min       0.000000     2.000000     1.000000     4.000000
25%    1049.500000     5.000000     6.000000    15.000000
50%    2099.000000     8.000000    12.000000    22.000000
75%    3148.500000    12.000000    26.000000    36.000000
max    4198.000000    32.000000   946.000000   961.000000
{'14784131 14858934 14784131 14845064': 0, '14852788 14717848 15639958 15632020': 1, '14844856 14724258 14925237 14854807': 2, '14925756 15639967 14853254 14728639': 3, '14844593 14924945': 4, '15709098 14716590 14924703 14779559': 5, '14726332 14728344 14854542 14844591': 6, '14858934 15636660 15704193 14849963': 7, '15710359 14847407 14845602 14859696': 8, '14794687 14782344': 9, '15630486 15702410 14718849 15709093': 10, '15632285 15706536 14721977 14925219': 11, '147829

## Train 5-Fold Model


In [4]:
log_steps = 10
save_steps = 20
kfold=5
tp = TrainParams(
    epoch_size=30,
    lr=1e-3,
    loss_fn=nn.CrossEntropyLoss(),
    max_seq_len=1000,
    batch_size=64,
    dropout_rate=0.5,
    label_size = len(label2idx),
    vocab_size = w2v.vocab_size,
    embedding_dim = w2v.embedding_size + c2v.embedding_size,
    embedding1=c2v.embedding, 
    embedding2 =w2v.embedding,
    filter_size=70,
    kernel_size_list = [2,3,4,5],
    hidden_size = 100,
    early_stop_params = {
        'monitor':'f1_micro',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    scheduler_params={'mode': 'max',
                     'factor': 0.3,
                     'patience': 1,
                     'verbose': True,
                     'threshold':0.0001,
                     'threshold_mode':'rel',
                     'cooldown':0,
                     'min_lr':1e-6},
    max_unsupervised=50,
    labeled_size = int(df.shape[0]/kfold *(kfold-1)),
    temporal_alpha = 0.6,
    ramp_up_method='sigmoid',
    spatial_dropout_rate=0.5
    
)



In [6]:
kf = KFold(n_splits=kfold, shuffle=True, random_state=24)
for fold,(train_index, valid_index) in enumerate(kf.split(df)):
    train, valid = df.iloc[train_index], df.iloc[valid_index]

    # combine label and unlabel data
    train_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               train['name'].values.tolist(),train['description'].values.tolist() ,
                               train['label'].values.tolist())
    unlabel_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,
                                 test['name'].values, test['description'].values, ['unlabel'] *test.shape[0])
    valid_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx, 
                               valid['name'].values, valid['description'].values, valid['label'].values)
    train_sampler = RandomSampler(train_dataset)
    unlabel_sampler = RandomSampler(unlabel_dataset)
    valid_sampler = SequentialSampler(valid_dataset)
    train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
    valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=int(2*tp.batch_size))
    unlabel_loader = DataLoader(unlabel_dataset, sampler=unlabel_sampler, batch_size=tp.batch_size)
    tp.update({'num_train_steps': len(train_loader)})
    
    CKPT = './checkpoint/textcnn_temporal_aug/k{}'.format(fold)
    saver = ModelSave(CKPT, continue_train=False)
    saver.init()
    tb = SummaryWriter(CKPT)
    es = EarlyStop(**tp.early_stop_params)
    global_step = 0
    model = TextcnnAugTemporal(tp)
    optimizer, scheduler = model.get_optimizer()
    
    for epoch_i in range(tp['epoch_size']):
        if global_step==1:
            print(model)
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
        print("-"*60)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()
        total_loss, batch_loss, batch_counts = 0, 0, 0

        model.train()
        unlabel_iter = iter(unlabel_loader)
        for step, batch in enumerate(train_loader):
            try:
                unlabel_batch = next(unlabel_iter)
            except:
                unlabel_iter = iter(unlabel_loader)
                unlabel_batch = next(unlabel_iter)
            global_step +=1
            batch_counts +=1

            #Forward propogate
            model.zero_grad()
            feature = {k:torch.cat([v, unlabel_batch[k]],dim=0).to(device) for k,v in batch.items()}
            logits = model(feature)
            loss= model.compute_loss(feature, logits)
            tb.add_scalar('loss/sup_loss', model.supervised_loss, global_step=global_step)
            tb.add_scalar('loss/mse_loss', model.mse_loss, global_step=global_step)
            tb.add_scalar('loss/avg_loss', loss, global_step=global_step)
            batch_loss += loss.item()
            total_loss += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            # Log steps for train loss logging
            if (step % log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                time_elapsed = time.time() - t0_batch
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

            # Save steps for ckpt saving and dev evaluation
            if (step % save_steps == 0 and step != 0) or (step == len(train_loader) - 1):
                val_metrics = multi_cls_metrics(model, valid_loader, device)
                for key, val in val_metrics.items():
                    tb.add_scalar(f'metric/{key}', val, global_step=global_step)
                avg_train_loss = total_loss / step
                tb.add_scalars('loss/train_valid',{'train': avg_train_loss,
                                                    'valid': val_metrics['val_loss']}, global_step=global_step)
                saver(total_loss / step, val_metrics['val_loss'], epoch_i, global_step, model, optimizer, scheduler)
        model.epoch_update()
        # On Epoch End: calcualte train & valid loss and log overall metrics
        time_elapsed = time.time() - t0_epoch
        val_metrics = multi_cls_metrics(model, valid_loader, device)
        avg_train_loss = total_loss / step
        scheduler.step(val_metrics['f1_micro'])
        print("-"*70)
        print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
        multi_cls_log(epoch_i, val_metrics)
        print("\n")
        if es.check(val_metrics):
            break

./checkpoint/textcnn_temporal_aug/k0 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   4.436095   |     -     |   28.51  
   1    |   20    |   2.846236   |     -     |   25.81  
   1    |   30    |   2.778615   |     -     |   31.50  
   1    |   40    |   2.543316   |     -     |   24.99  
   1    |   50    |   2.397907   |     -     |   31.47  
   1    |   52    |   2.279352   |     -     |   4.57   
----------------------------------------------------------------------
   1    |    -    |   3.058009   |  2.273164  |  153.05  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   1    |  10.784%  |  67.643%  |  23.120%  |     8.581%      |   10.784%    |  31.429%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F

   8    |   30    |   0.820263   |     -     |   22.08  
   8    |   40    |   0.814040   |     -     |   18.69  
   8    |   50    |   0.766343   |     -     |   23.75  
   8    |   52    |   0.645983   |     -     |   3.39   
----------------------------------------------------------------------
   8    |    -    |   0.979550   |  1.039277  |  113.50  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   8    |  50.948%  |  83.895%  |  59.020%  |     62.806%     |   50.948%    |  70.357%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   8    |  70.357%  |     -     |     -     |     70.357%     |   70.357%    |  70.357%  




 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
--------------------------------------

KeyboardInterrupt: 

./checkpoint/textcnn_temporal/k0 model cleaned
 Epoch  |  Batch  |  Train Loss  |  Val Loss   |  Elapsed 
------------------------------------------------------------
   1    |   10    |   3.989696   |     -     |   15.96  
   1    |   20    |   2.422722   |     -     |   14.57  


C:\Users\86188\.conda\envs\py36_tf23\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\86188\.conda\envs\py36_tf23\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\86188\.conda\envs\py36_tf23\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
C:\Users\86188\.conda\envs\py36_tf23\lib\site-packages\torchmetrics\functional\classification\average_precision.py:168: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these 

   1    |   30    |   2.683932   |     -     |   21.76  
   1    |   40    |   2.398114   |     -     |   15.46  
   1    |   50    |   2.244433   |     -     |   19.81  
   1    |   60    |   2.094535   |     -     |   12.71  
   1    |   70    |   1.922081   |     -     |   18.63  
   1    |   80    |   1.842103   |     -     |   12.38  
----------------------------------------------------------------------
   1    |    -    |   2.499573   |  1.815674  |  137.35  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   1    |  21.510%  |  72.734%  |  30.271%  |     17.847%     |   21.510%    |  44.167%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------------------------------------------
   1    |  44.167%  |     -     |     -     |     44.167%     |   44.167%    

   8    |   10    |   0.325137   |     -     |   14.28  
   8    |   20    |   0.420930   |     -     |   12.76  
   8    |   30    |   0.208133   |     -     |   18.44  
   8    |   40    |   0.253256   |     -     |   12.32  
   8    |   50    |   0.181493   |     -     |   18.45  
   8    |   60    |   0.242700   |     -     |   12.77  
   8    |   70    |   0.196891   |     -     |   18.60  
   8    |   80    |   0.141491   |     -     |   12.28  
----------------------------------------------------------------------
   8    |    -    |   0.250318   |  1.203201  |  125.63  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
   8    |  56.313%  |  85.488%  |  61.258%  |     65.607%     |   56.313%    |  72.976%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------

  15    |   10    |   0.293777   |     -     |   14.18  
  15    |   20    |   0.256033   |     -     |   12.85  
  15    |   30    |   0.119604   |     -     |   18.57  
  15    |   40    |   0.131648   |     -     |   12.71  
  15    |   50    |   0.123744   |     -     |   18.60  
  15    |   60    |   0.122728   |     -     |   13.17  
  15    |   70    |   0.133628   |     -     |   18.65  
  15    |   80    |   0.117976   |     -     |   12.65  
----------------------------------------------------------------------
  15    |    -    |   0.166064   |  1.349700  |  127.31  


 Epoch  | Macro Acc | Macro AUC | Macro AP  | Macro Precision | Macro Recall | Macro F1 
------------------------------------------------------------------------------------------
  15    |  57.254%  |  85.525%  |  61.909%  |     68.597%     |   57.254%    |  73.571%  
 Epoch  | Micro Acc | Micro AUC | Micro AP  | Micro Precision | Micro Recall | Micro F1 
------------------------------------------------------

## K-Fold Evalutaion

In [8]:
test = test_process()
test_dataset = MixDataset(tp.max_seq_len, w2v, c2v, phraser, label2idx,  test['name'].values, test['description'].values)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [ ]:
result = kfold_inference(test_loader, tp, Textcnn, './checkpoint/textcnn_temporal', 5, device)
result['pred'] = result['pred_avg']
result_process(result, label2idx, './submit/textcnn_temporal_5fold_avg.csv')